# BUILD THE NEURAL NETWORK

Neural networks comprise of layers/modules that perform operations on data. The `torch.nn` namespace provides all the building blocks you need to build 
your own neural network. Every module in PyTorch subclasses the `nn.Module`. A neural network is a module itself that consists of other modules (layers). 
This nested structure allows for building and managing complex architectures easily.

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [4]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


## Define the class
We define our neural network by subclassing `nn.Module`, and initialize the neural network layers in `__init__`. Every `nn.Module` subclass implements the operations on input data in the forward method.

In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model’s forward, along with some background operations. **Do not call `model.forward()` directly!**

Calling the model on the input returns a 2-dimensional tensor with dim=0 corresponding to each output of 10 raw predicted values for each class, 
and dim=1 corresponding to the individual values of each output. 
We get the prediction probabilities by passing it through an instance of the `nn.Softmax` module.


In [18]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
print(f"logits {logits}")
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

logits tensor([[-0.0419, -0.0241,  0.1340,  0.0301,  0.0066, -0.0381,  0.0573,  0.0994,
         -0.0561, -0.0007]], device='mps:0', grad_fn=<LinearBackward0>)
Predicted class: tensor([2], device='mps:0')


## Model Layers

Let’s break down the layers in the FashionMNIST model. To illustrate it, we will take a sample minibatch of 3 images of size 28x28 and see what happens to it as we pass it through the network.

In [20]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


### nn.Flatten

We initialize the `nn.Flatten` layer to convert each `2D 28x28` image into a contiguous array of 784 pixel values (the minibatch dimension (at dim=0) is maintained).


In [21]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


### nn.Linear

The linear layer is a module that applies a linear transformation on the input using its stored weights and biases.

In [22]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### nn.ReLU

Non-linear activations are what create the complex mappings between the model’s inputs and outputs. 
They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model.

In [24]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 6.2961e-01, -1.2621e-01, -3.9469e-01,  2.3539e-01,  3.8986e-01,
         -9.0424e-02,  3.7933e-02, -1.1876e-01,  2.4769e-01,  1.5144e-01,
         -1.8520e-01,  1.1601e-03, -4.0249e-01,  2.7954e-01,  9.3056e-02,
          1.1420e-01,  4.2645e-01, -4.0764e-01,  3.1598e-01, -5.6935e-02],
        [ 8.8202e-01, -4.8923e-02, -5.6764e-01, -5.4005e-02, -7.3714e-03,
         -2.1598e-01, -1.8244e-01, -8.6752e-02,  7.8759e-02, -2.7725e-01,
         -1.8436e-01,  1.9359e-01, -5.6556e-01,  2.0110e-01, -1.1829e-01,
         -8.7833e-03,  3.7301e-01, -3.4590e-01,  3.0682e-01, -1.5499e-01],
        [ 2.5317e-01, -1.8795e-01, -3.1644e-01,  9.0782e-03,  4.7826e-01,
         -2.4548e-01, -2.5391e-01, -5.2906e-02,  8.9920e-05, -6.9755e-02,
         -3.8659e-01,  1.1220e-01, -5.3168e-01,  3.1426e-01, -1.7867e-01,
          1.8152e-01,  2.5654e-01, -4.6630e-01,  5.6413e-01,  2.1538e-01]],
       grad_fn=<AddmmBackward0>)


After ReLU: tensor([[6.2961e-01, 0.0000e+00, 0.0000e+00, 2.3

### nn.Sequential

nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. 
You can use sequential containers to put together a quick network like seq_modules.

In [29]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
logits.size()

torch.Size([3, 10])

### nn.Softmax

The last linear layer of the neural network returns logits - raw values in [-infty, infty] - which are passed to the nn.Softmax module. The logits are scaled to values [0, 1] representing the model’s predicted probabilities for each class. dim parameter indicates the dimension along which the values must sum to 1.

In [31]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.0846, 0.0922, 0.1139, 0.0663, 0.1131, 0.0950, 0.1083, 0.0876, 0.1104,
         0.1285],
        [0.0662, 0.0891, 0.0994, 0.0722, 0.1142, 0.1064, 0.1152, 0.0860, 0.1298,
         0.1213],
        [0.0833, 0.0996, 0.1072, 0.0761, 0.1222, 0.0920, 0.1126, 0.0950, 0.0957,
         0.1162]], grad_fn=<SoftmaxBackward0>)

## Model Parameters

Many layers inside a neural network are parameterized, i.e. have associated weights and biases that are optimized during training. 
Subclassing `nn.Module` automatically tracks all fields defined inside your model object, and makes all parameters accessible using your model’s `parameters()` or `named_parameters()` methods.



In [32]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0044, -0.0135, -0.0091,  ..., -0.0240, -0.0225,  0.0221],
        [ 0.0179, -0.0227, -0.0324,  ...,  0.0163,  0.0066,  0.0351]],
       device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0041, -0.0297], device='mps:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0088,  0.0212,  0.0377,  ..., -0.0255,  0.0183,  0.0012],
        [ 0.0258, -0.0054,  0.0411,  ..., -0.0108,  0.0058,  0.0194]],
       device='mps:0', grad_fn=<Slice